In [6]:
import cv2


video_path = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV"

cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', height=760, width=1366) 

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255,255,255)
lineType = 2

cap = cv2.VideoCapture(video_path)
frame_count = -1

if (cap.isOpened()== False): 
    print("check the videopath")

while(cap.isOpened()):

    ret, frame = cap.read()
    key = cv2.waitKey(25)
    
    if(ret == True):
        
        frame_count = frame_count + 1
        
        if key == ord('p'): #pause a video

            while True:
                
                key2 = cv2.waitKey(25)

                cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                cv2.imshow('frame', frame)
                
                if key2 == ord('s'): #start after pausing
                    print(frame_count)
                    break
        
        if key == ord('q'): #quit a video
            break
        
        cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
        cv2.imshow('frame', frame)

    else:
        break


cap.release()
cv2.destroyAllWindows()





43
80
91
100
162


In [4]:
import cv2


video_path = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV"

cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', height=760, width=1366)

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255,255,255)
lineType = 2

cap = cv2.VideoCapture(video_path)
frame_count = -1

ss = []
se = []
l = []

if (cap.isOpened()== False): 
    print("check the videopath")

while(cap.isOpened()):

    ret, frame = cap.read()
    key = cv2.waitKey(100)
    
    if(ret == True):
        
        frame_count = frame_count + 1
        
        cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
        cv2.imshow('frame', frame)
        
        if key == ord('p'): #pause a video

            ss.append(frame_count)
            l.append(frame_count)
            
        if key == ord('s'): #pause a video

            se.append(frame_count)
            l.append(frame_count)
        
        if key == ord('q'): #quit a video
            break
        
        

    else:
        break


cap.release()
cv2.destroyAllWindows()

print(ss)
print(se)
print(l)



[28, 38, 43]
[34, 40, 45]
[28, 34, 38, 40, 43, 45]
